Setup model and input files to create SVIP like connectomes in ABIDE

In [1]:
import os
import numpy as np
import pandas as pd
import patsy as pat
from scipy import io as sio

In [2]:
# Paths
model_p = '/remote/guillimin/simdata/data/abide_1/pheno/abide_1_psm_sample.csv'
comp_p = '/remote/guillimin/simdata/simons_vip/pheno/phenotype_all_both_sessions_norm.csv'
data_p = '/remote/guillimin/simdata/data/abide_1/preprocessed/niak_gs0_lp1/'
remote_p = '/gs/project/gsf-624-aa/data/abide_1/preprocessed/niak_gs0_lp1'
f_temp = 'fmri_{:07}_session_1_run1.nii.gz'
out_p = '/remote/guillimin/simdata/data/abide_1/connectome'
if not os.path.isdir(out_p):
    os.makedirs(out_p)

In [3]:
model = pd.read_csv(model_p)
biv = model[~model.biv.isnull()]
biv_c = biv[biv.DX_GROUP=='Control']
biv_c['niak_sub'] = 's' + biv['SUB_ID'].astype(str)
comp = pd.read_csv(comp_p)

/home/surchs/Venv/py35/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
glm_model = pat.dmatrix('0+SITE_ID+FD_scrubbed', data=biv_c, return_type='dataframe')
contrasts = {c:0 for c in glm_model.columns[:-2]}
contrasts['AGE_AT_SCAN'] = 1
contrasts['FD_scrubbed'] = 0
# Add subject ID and move to front
glm_model['']=biv_c['niak_sub']
ord_list = [''] + list(glm_model.columns)
glm_model = glm_model[ord_list]

In [5]:
glm_model.head()

,,SITE_ID[KKI],SITE_ID[LEUVEN_1],SITE_ID[MAX_MUN],SITE_ID[NYU],SITE_ID[PITT],SITE_ID[TRINITY],SITE_ID[UCLA_1],SITE_ID[UM_1],SITE_ID[USM],SITE_ID[YALE],FD_scrubbed,
0,s50772,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.133364,s50772
1,s50774,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153141,s50774
2,s50776,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218636,s50776
3,s50777,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159246,s50777
4,s50779,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.184077,s50779


In [6]:
data = {'fmri':
        {row['niak_sub']:
         {'s1':
          {'r1':os.path.join(remote_p, f_temp.format(row['SUB_ID']))}} 
         for rid, row in biv_c.iterrows()}
       }

In [7]:
# Create the model
out_model_p = os.path.join(out_p, 'abide_glm_model.csv')
# Save the model
glm_model.to_csv(out_model_p, index=False)

In [11]:
data['model'] = os.path.join('/gs/project/gsf-624-aa/data/abide_1/svip_connectome', 'abide_glm_model.csv')
data['opt'] = {'fdr':0.05,
               'test':{'biv_effect':{'group':{'contrast':contrasts}}},
               'folder_out':os.path.join('/gs/project/gsf-624-aa/data/abide_1/glm', 'abide_biv_regressed')}

In [12]:
# Save the data
sio.savemat(os.path.join(out_p, 'data.mat'), data)

In [13]:
out_p

'/remote/guillimin/simdata/data/abide_1/svip_connectome'